<a href="https://colab.research.google.com/github/cifkao/confugue/blob/colab/docs/pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confugue

- Installation: `pip install confugue`
- Docs: [confugue.readthedocs.io](https://confugue.readthedocs.io/)
- Code: [github.com/cifkao/confugue](https://github.com/cifkao/confugue)

Confugue is a **hierarchical configuration framework** for Python. It provides a wrapper class for **nested configuration dictionaries** (usually loaded from YAML files), which can be used to easily configure complicated object hierarchies.

This notebook is intended as a quick start guide for **deep learning** users. It uses PyTorch for example purposes, but it should be easy to follow even for people working with other frameworks like TensorFlow. It should also be stressed that Confugue is in no way limited to deep learning applications, and a [getting started guide](https://confugue.readthedocs.io/en/latest/general-guide.html) for general Python users is available.

In [0]:
!pip install confugue

## Basic PyTorch example
We are going to start with a basic PyTorch model, adapted from the [CIFAR-10 tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html). First, let's see how we would code the model *without* using Confugue.

In [0]:
import torch
from torch import nn

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = x.flatten(start_dim=1)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

### Making it configurable
Instead of hard-coding all the hyperparameters like above, we want to be able to specify them in a configuration file. To do so, we are going to decorate our class with the `@configurable` decorator. This provides it with a magic `_cfg` property, giving it access to the configuration. We can then rewrite our `__init__` as follows:

In [0]:
from confugue import configurable, Configuration

In [0]:
@configurable
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = self._cfg['conv1'].configure(nn.Conv2d, in_channels=3)
        self.conv2 = self._cfg['conv2'].configure(nn.Conv2d)
        self.pool = self._cfg['pool'].configure(nn.MaxPool2d)
        self.fc1 = self._cfg['fc1'].configure(nn.Linear)
        self.fc2 = self._cfg['fc2'].configure(nn.Linear, out_features=10)
        self.act = self._cfg['act'].configure(nn.ReLU)

    def forward(self, x):
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = x.flatten(start_dim=1)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

Instead of creating each layer directly, we configure it with values from the corresponding section of the configuration file (which we will see in a moment). Notice that we can still specify arguments in the code (e.g. `in_channels=3` for the `conv1` layer), but these are treated as defaults and can be overridden in the configuration file if needed.

### Loading configuration from a YAML file
Calling `Net()` directly would result in an error, since we haven't specified defaults for all the required parameters of each layer.
We therefore need to create a configuration file to supply them:

In [0]:
%%writefile config.yaml
conv1:
  out_channels: 6
  kernel_size: 5
conv2:
  in_channels: 6
  out_channels: 16
  kernel_size: 5
pool:
  kernel_size: 2
  stride: 2
fc1:
  in_features: 400
  out_features: 120
fc2:
  in_features: 120

# Note that we do not need to include the activation function ('act'), since it does not have any
# required parameters. We could, however, override the type of the activation function itself
# as follows:

# act:
#   class: !!python/name:torch.nn.Tanh

Writing config.yaml


We are now ready to load the file into a `Configuration` object and use it to configure our network:

In [0]:
cfg = Configuration.from_yaml_file('config.yaml')
cfg

Configuration({'conv1': {'out_channels': 6, 'kernel_size': 5}, 'conv2': {'in_channels': 6, 'out_channels': 16, 'kernel_size': 5}, 'pool': {'kernel_size': 2, 'stride': 2}, 'fc1': {'in_features': 400, 'out_features': 120}, 'fc2': {'in_features': 120}})

In [0]:
cfg.configure(Net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (act): ReLU()
)

## Nested configurables
One of the most useful features of Confugue is that `@configurable` classes and functions can use other configurables, and the structure of the configuration file will naturally follow this hierarchy. To see this in action, we are going to write a configurable `main` function which trains our simple model on the CIFAR-10 dataset.

In [0]:
import torchvision
from torchvision import transforms

@configurable
def main(num_epochs=1, log_period=2000, *, _cfg):
    net = _cfg['net'].configure(Net)
    criterion = _cfg['loss'].configure(nn.CrossEntropyLoss)
    optimizer = _cfg['optimizer'].configure(torch.optim.SGD, params=net.parameters(), lr=0.001)

    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                              transform=transform)
    train_loader = _cfg['data_loader'].configure(torch.utils.data.DataLoader, dataset=train_data,
                                                 batch_size=4, shuffle=True, num_workers=2)
  
    for epoch in range(num_epochs):
        for i, batch in enumerate(train_loader):
            inputs, labels = batch
            optimizer.zero_grad()
            loss = criterion(net(inputs), labels)
            loss.backward()
            optimizer.step()

            if (i + 1) % log_period == 0:
                print(i + 1, loss.item())

In [0]:
%%writefile config.yaml
net:
  conv1:
    out_channels: 6
    kernel_size: 5
  conv2:
    in_channels: 6
    out_channels: 16
    kernel_size: 5
  pool:
    kernel_size: 2
    stride: 2
  fc1:
    in_features: 400
    out_features: 120
  fc2:
    in_features: 120

optimizer:
  class: !!python/name:torch.optim.Adam
data_loader:
  batch_size: 8
num_epochs: 2
log_period: 1000

Overwriting config.yaml


In [0]:
cfg = Configuration.from_yaml_file('config.yaml')
cfg.configure(main)

Extracting ./data/cifar-10-python.tar.gz to ./data

1000 2.183788537979126
2000 2.050525665283203
3000 1.366685390472412
4000 1.4100595712661743
5000 1.7879905700683594
6000 1.708138108253479
1000 1.5253592729568481
2000 1.24685800075531
3000 1.349258303642273
4000 1.4699578285217285
5000 1.6091036796569824
6000 0.9718704223632812


## Configuring lists
The `configure_list` method allows us to configure a list of objects, with the parameters for each supplied from the configuration file. We are going to use this, in conjunction with `nn.Sequential`, to fully specify the model in the configuration file, so we won't need our `Net` class anymore.

In [0]:
%%writefile config.yaml
layers:
  - class: !!python/name:torch.nn.Conv2d
    in_channels: 3
    out_channels: 6
    kernel_size: 5
  - class: !!python/name:torch.nn.ReLU
  - class: !!python/name:torch.nn.MaxPool2d
    kernel_size: 2
    stride: 2
  - class: !!python/name:torch.nn.Conv2d
    in_channels: 6
    out_channels: 16
    kernel_size: 5
  - class: !!python/name:torch.nn.ReLU
  - class: !!python/name:torch.nn.MaxPool2d
    kernel_size: 2
    stride: 2
  - class: !!python/name:torch.nn.Flatten
  - class: !!python/name:torch.nn.Linear
    in_features: 400
    out_features: 120
  - class: !!python/name:torch.nn.ReLU
  - class: !!python/name:torch.nn.Linear
    in_features: 120
    out_features: 10

Writing config.yaml


Creating the model then becomes a matter of two lines of code:

In [0]:
cfg = Configuration.from_yaml_file('config.yaml')
nn.Sequential(*cfg['layers'].configure_list())

Sequential(
  (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten()
  (7): Linear(in_features=400, out_features=120, bias=True)
  (8): ReLU()
  (9): Linear(in_features=120, out_features=10, bias=True)
)

This offers a lot of flexibility, but it should be used with care. If your configuration file is longer than your code, you might be overusing it.

## Further reading
Confugue offers a couple more useful features, which are described [in the documentation](https://confugue.readthedocs.io/en/latest/more-features.html). You can also check out the [API reference](https://confugue.readthedocs.io/en/latest/api.html).